In [ ]:
import healpy
from cora.util import hputil, coord
import numpy as np
from matplotlib import pyplot as plt
from ch_util import ephemeris as ephem
from caput.time import unix_to_skyfield_time
import h5py
import time
from glob import glob
from drift.telescope import cylbeam

%matplotlib inline
plt.rcParams.update({'figure.figsize': (16, 12), 'font.size': 16})

In [ ]:
haslam_dsds = healpy.fitsfunc.read_map("./lambda_haslam408_dsds.fits")
haslam_nofilt = healpy.fitsfunc.read_map("./lambda_haslam408_nofilt.fits")

In [ ]:
hputil.coord_g2c(haslam_nofilt)

In [ ]:
nside = 512

In [ ]:
healpy.visufunc.mollview(haslam_nofilt)

# Convolve EW beam with Haslam

In [ ]:
beam_ew = cylbeam.fraunhofer_cylinder(lambda x: np.ones_like(x), 20.)

In [ ]:
plt.plot(beam_ew(np.linspace(-np.pi/2, np.pi/2, 800))**2)

In [ ]:
ang_pos = healpy.pix2ang(nside, np.arange(len(haslam_nofilt)))[1]
ang_pos *= -1
ang_pos[np.where(ang_pos + np.pi < 0)] += 2* np.pi

In [ ]:
beam_win = beam_ew(ang_pos)

In [ ]:
healpy.visufunc.mollview(beam_win)

In [ ]:
cart_proj = healpy.projector.CartesianProj()
cart_map = cart_proj.projmap(haslam_nofilt, lambda x,y,z: healpy.vec2pix(nside, x, y, z))
xy = cart_proj.ij2xy()

In [ ]:
zenith = np.radians((90. - ephem.CHIMELATITUDE, 0.))
rot = (0., 0., 0.)
angpos = np.radians((90. - xy[1].data.flatten(), xy[0].data.flatten())).T
width = 20.

In [ ]:
def get_beam_amp(angpos, zenith, rot=(0.,0.,0.), width=20., ns_beam=False):

    that, phat = coord.thetaphi_plane_cart(zenith)

    xhat, yhat, zhat = cylbeam.rotate_ypr(rot, phat, -that, coord.sph_to_cart(zenith))

    xplane = lambda t: np.ones_like(t)
    yplane = lambda t: np.cos(t)
    #yplane = lambda t: cylbeam.beam_exptan(t, 1.)

    beampat = cylbeam.fraunhofer_cylinder(xplane, width)

    cvec = coord.sph_to_cart(angpos)
    horizon = (np.dot(cvec, coord.sph_to_cart(zenith)) > 0.0).astype(np.float64).reshape(angpos.shape[0])

    ew_amp = beampat(np.dot(cvec, xhat)).reshape(angpos.shape[0])
    if ns_beam:
        ns_amp = yplane(np.arcsin(np.dot(cvec, yhat))).reshape(angpos.shape[0])
    else:
        ns_amp = 1.

    return ew_amp, ns_amp, horizon

def convolve_beam_cart(in_map, beam_map):
    
    fft_map = np.fft.fft(in_map, axis=1)
    fft_win = np.fft.fft(beam_map, axis=1)

    return np.fft.fftshift(np.fft.ifft(fft_map * fft_win, axis=1), axes=(1,)).real

#norm = np.sum(beam_amp**2 * horizon, axis=1)
#norm[np.where(norm == 0.)] = 1.

#beam_amp /= norm[:,np.newaxis]

In [ ]:
beam_amp = get_beam_amp(angpos, zenith, ns_beam=True)

In [ ]:
plt.imshow((beam_amp[0]**2 * beam_amp[2]).reshape(xy[0].data.shape), origin='lower', aspect='auto')
plt.colorbar()

In [ ]:
haslam_smth = convolve_beam_cart(cart_map, beam_amp**2)

In [ ]:
plt.imshow(np.log10(np.abs(cart_map)), origin='lower', aspect='auto',
           extent=(xy[0].data[0,0], xy[0].data[0,-1], xy[1].data[0,0], xy[1].data[-1,0]))

In [ ]:
plt.imshow(np.log10(np.abs(haslam_smth)), origin='lower', aspect='auto', vmin=3)
plt.colorbar()

# Now in HEALpix

In [ ]:
rings = healpy.ringinfo(nside, np.arange(4*nside))

In [ ]:
haslam_smth = np.zeros_like(haslam_nofilt)
beam_map = np.zeros_like(haslam_nofilt)
for r in xrange(len(rings[0])):
    pix = np.arange(rings[0][r], rings[0][r] + rings[1][r])
    if len(pix) == 0:
        continue
    angpos = np.array(healpy.pix2ang(nside, pix, nest=False)).T
    beam_amp = get_beam_amp(angpos, zenith, ns_beam=True)
    fft_map = np.fft.fft(haslam_nofilt[pix])
    fft_beam = np.fft.fft(beam_amp**2)
    
    beam_map[pix] = beam_amp
    #haslam_smth[pix] = np.fft.fftshift(np.fft.ifft(fft_map*fft_beam)).real
    haslam_smth[pix] = np.fft.ifft(fft_map*fft_beam).real

In [ ]:
fig = plt.figure()
healpy.visufunc.mollview(np.log10(haslam_nofilt), nest=False, fig=1, xsize=2000)

In [ ]:
fig = plt.figure()
healpy.visufunc.mollview(np.log10(haslam_smth), nest=False, fig=1, xsize=2000)

plt.savefig("convolved_beam_haslam_no_ns.pdf", dpi=300, bbox_to_inches='tight')

In [ ]:
fig = plt.figure()
healpy.visufunc.mollview(beam_map, nest=False, fig=1, xsize=2000)

## TODO: 
- Do correct convolution to get smilies
- Look into whether theta direction is getting flipped.

In [ ]:
np.arange(20).reshape(10,2).T

In [ ]:
np.array(healpy.pix2ang(nside,(1000,1002, 1003)))

## Instead, do the integration/convolution at every step

In [ ]:
healpy.rotator.euler_matrix_new?

In [ ]:
ra = np.radians(90.)
lat = np.radians(ephem.CHIMELATITUDE)
r = healpy.rotator.Rotator((np.pi/2-ra, -lat, -np.pi/2), eulertype='X', deg=False)

In [ ]:
haslam_telcoord = healpy.pix2ang(nside, np.arange(haslam_nofilt.shape[0]))[1]
haslam_telcoord = r.rotate_map(haslam_telcoord)

In [ ]:
fig = plt.figure()
healpy.visufunc.mollview(haslam_telcoord, nest=False, fig=1, xsize=2000)

In [ ]:
test_map = np.zeros((400, 800))
ra = np.linspace(0., 2*np.pi, test_map.shape[1])
dec = np.linspace(0., np.pi, test_map.shape[0])

rot = healpy.rotator.Rotator((np.pi/2, -lat, np.pi/2), eulertype='X', deg=False)
tel_beam = rot.rotate_map(beam_map)

#ring_theta = np.arcsin(rings[2])

#for i in range(test_map.shape[0]):
#    for j in range(test_map.shape[1]):
#        r = 
#        pix = np.arange(rings[0][r], rings[0][r] + rings[1][r])
#        test_map[i,j] = np.sum()

In [ ]:
fig = plt.figure()
healpy.visufunc.mollview(tel_beam, nest=False, fig=1, xsize=2000)

In [ ]:
plt.plot(rings[2])